<a href="https://colab.research.google.com/github/gabilodeau/INF8770/blob/master/Codage%20arithmetique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INF8770 Technologies multimédias

Polytechnique Montréal

Exemple de codage arithmétique

In [1]:
import numpy as np

Message à coder. On codera seulement les SymbACoder premiers symboles. Limite des float. Sinon, on peut utiliser des bigfloat. 

In [2]:
Message = "ABAABCABACABBABCDAADAAABABAAABAABBABABAABAAB"
SymbACoder = 6

Calcul des probabilités de chaque symbole.

In [3]:
ProbSymb =[[Message[0], Message.count(Message[0])/len(Message)]]
nbsymboles = 1

for i in range(1,len(Message)):
    if not list(filter(lambda x: x[0] == Message[i], ProbSymb)):
        ProbSymb += [[Message[i], ProbSymb[-1][1]+Message.count(Message[i])/len(Message)]]
        nbsymboles += 1
        
longueurOriginale = np.ceil(np.log2(nbsymboles))*SymbACoder 

In [ ]:
print(ProbSymb)

[['A', 0.5454545454545454], ['B', 0.8863636363636362], ['C', 0.9545454545454544], ['D', 0.9999999999999998]]


Codage du message. On divise successivement l'intervalle représentant le symbole à coder.

In [4]:
Code = ProbSymb[:]
Code = [['', 0]] + ProbSymb[:]
print(Code)
for i in range(SymbACoder): 
    #Cherche dans quel intervalle est le symbole à coder
    temp = list(filter(lambda x: x[0] == Message[i], Code))
    indice = Code.index(temp[0])

    #Calcul des bornes pour coder le caractère
    Debut = Code[indice-1][1]
    Plage = Code[indice][1] - Debut
    print(Message[i], ' est dans l\'intervalle', indice, ' de ', Debut, ' à ', Debut + Plage)
    print()      
    #Nouveaux intervalles pour coder le prochain symbole
    Code = [['', Debut]]  
    for j in range(len(ProbSymb)):
        Code += [[ProbSymb[j][0], Debut+ProbSymb[j][1]*Plage]]
          
    print(Code)

[['', 0], ['A', 0.5454545454545454], ['B', 0.8863636363636362], ['C', 0.9545454545454544], ['D', 0.9999999999999998]]
A  est dans l'intervalle 1  de  0  à  0.5454545454545454

[['', 0], ['A', 0.29752066115702475], ['B', 0.4834710743801652], ['C', 0.5206611570247932], ['D', 0.5454545454545453]]
B  est dans l'intervalle 2  de  0.29752066115702475  à  0.4834710743801652

[['', 0.29752066115702475], ['A', 0.3989481592787377], ['B', 0.4623403456048083], ['C', 0.4750187828700224], ['D', 0.4834710743801651]]
A  est dans l'intervalle 1  de  0.29752066115702475  à  0.3989481592787377

[['', 0.29752066115702475], ['A', 0.35284475104159546], ['B', 0.38742230721945214], ['C', 0.39433781845502347], ['D', 0.3989481592787377]]
A  est dans l'intervalle 1  de  0.29752066115702475  à  0.35284475104159546

[['', 0.29752066115702475], ['A', 0.3276974374576997], ['B', 0.3465579226456215], ['C', 0.3503300196832059], ['D', 0.35284475104159546]]
B  est dans l'intervalle 2  de  0.3276974374576997  à  0.3465579

On choisit une valeur dans l'intervalle final. On cherche la valeur qui prend le moins de bits. On fait la somme de puissance négative de 2. Permet aussi d'obtenir le code binaire.

In [5]:
ok = True
valfinale = 0
valEnBits = list('')
p = 0
while ok:
    p += 1
    #Essayer différentes sommes de puissance négative de 2
    valfinale += np.power(2.0,-p)
    valEnBits += '1' 
    if valfinale >= (Debut + Plage):
        valfinale -= np.power(2.0,-p) #Hors de la borne maximale, on annule l'ajout.
        valEnBits[-1] ='0'
    elif valfinale >= Debut :
        ok = False

print(valfinale)
print("".join(valEnBits))

0.3447265625
0101100001


Autre façon de convertir en bits qui permet de voir l'importance du choix de la valeur finale.

In [6]:
#Table hex vers binaire
hex2bin = dict('{:x} {:04b}'.format(x,x).split() for x in range(16))

def float_dec2bin(d):
    
    #Note: je ne suis pas sûr que cela fonctionne toujours... J'ai fait un nombre de tests limité.
    
    hx = float(d).hex() #Conversion float vers hex
    p = hx.index('p')
    #Conversion hex vers bin avec la table
    bn = ''.join(hex2bin.get(char, char) for char in hx[2:p])
    code = list(bn)
    indice = code.index('.') # position du séparateur des décimales
    puissance = int(hx[p+2:]) # Décalage
    if puissance >= indice:
        #On ajoute des zéros pour pouvoir décaler le séparateur des décimales.
        zerosdeplus = "0" * (puissance-indice+1)
        bn = zerosdeplus + bn
        code = list(bn)
        indice = code.index('.') # nouvelle position du séparateur des décimales

    print(''.join(code))    
    
    #Décalage du séparateur décimal selon la puissance    
    for i in range(0,puissance):
        temp = code[indice-i-1];
        code[indice-i-1] = code[indice-i]
        code[indice-i] = temp
     
    print(''.join(code))
    
    #Enlève les zéros de trop et la partie avant le séparateur décimal
    
    ind = code.index('.')+1
    code = code[ind:]
    ind= code[::-1].index('1')
    code = code[:(len(code)-ind)]   
    codebinaire = ''.join(code)

    return codebinaire

messagecode = float_dec2bin(valfinale) #Essayer d'autres valeurs qui tombent dans l'intervalle
print(messagecode)


0001.0110000100000000000000000000000000000000000000000000
00.010110000100000000000000000000000000000000000000000000
0101100001


Longueur en bits du message codé et celle de l'original

In [7]:
longueur = len(messagecode)
print("Longueur = {0}".format(longueur))
print("Longueur originale = {0}".format(longueurOriginale))

Longueur = 10
Longueur originale = 12.0
